In [39]:
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly


mpl.rcParams['figure.figsize']=(16,9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

import plotly.graph_objects as go

In [40]:

df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,"Korea, South"
878,2022-06-18,17844905,86230982,12563399,27204955,18276552
879,2022-06-19,17879160,86246101,12563399,27211896,18280090
880,2022-06-20,17896065,86297081,12563399,27334993,18289373
881,2022-06-21,17959329,86452232,12613634,27454225,18298341
882,2022-06-22,18014202,86636306,12613634,27573585,18305783


In [41]:
df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,"Korea, South"
878,2022-06-18,17844905,86230982,12563399,27204955,18276552
879,2022-06-19,17879160,86246101,12563399,27211896,18280090
880,2022-06-20,17896065,86297081,12563399,27334993,18289373
881,2022-06-21,17959329,86452232,12613634,27454225,18298341
882,2022-06-22,18014202,86636306,12613634,27573585,18305783


In [42]:
country_list = df_analyse.columns[1:]

In [43]:
def quick_plot(x_in, df, y_scale = 'log', slider = False, xname = ' ', yname = ' ', figname = ' '):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
        xname : str
            Title of the X axis
        yname : str
            Title of the Y axis
        figname : str
            Title of the figure
        
        Returns:
        ----------
        Return type: Figure
    
    """    
    fig = go.Figure()
    for each in df.columns:
        fig.add_trace(go.Scatter(x=x_in, 
                                 y=df[each], 
                                 name= each,
                                 mode= 'markers+lines',
                                 line_width = 1,
                                 marker_size = 3)
                     )
    fig.update_layout(xaxis_title = xname,
                      yaxis_title = yname,
                      width = 1000, 
                      height = 800,
                      title={
                        'text': figname,
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'}
                    )
    fig.update_yaxes(type = y_scale)
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )    
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [44]:
#to check the helper function
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='log',
           slider=True,
           xname = 'Date',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Corona confirmed cases'
          )

In [45]:
# Compare confirmed cases from some 'threshold' value
threshold = 150 

In [46]:
compare_list = []
for pos, each in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[each][df_analyse[each]>threshold]))  

In [47]:
pd_sync_timelines = pd.DataFrame(compare_list, index = df_analyse.columns[1:]).T

In [48]:
pd_sync_timelines['day'] = np.arange(pd_sync_timelines.shape[0])

In [49]:
pd_sync_timelines.head()

,Italy,US,Spain,Germany,"Korea, South",day
0,155.0,184.0,165.0,188.0,204.0,0
1,229.0,237.0,222.0,240.0,433.0,1
2,322.0,403.0,259.0,349.0,602.0,2
3,453.0,519.0,400.0,534.0,833.0,3
4,655.0,594.0,500.0,684.0,977.0,4


In [50]:
quick_plot(pd_sync_timelines.day,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True,
           xname = 'Time in days',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Comparison of corona cases'
          )

# Doubling Rate

In [51]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [52]:
max_days=100

In [53]:
norm_slopes={
    'doubling every day':doubling_rate(100,np.arange(max_days),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}

In [54]:
pd_sync_timelines_w_slope = pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)
pd_sync_timelines_w_slope.head()

,doubling every day,doubling every two days,doubling every 4 days,doubling every 10 days,Italy,US,Spain,Germany,"Korea, South",day
0,100.0,100.000000,100.000000,100.000000,155.0,184.0,165.0,188.0,204.0,0
1,200.0,141.421356,118.920712,107.177346,229.0,237.0,222.0,240.0,433.0,1
2,400.0,200.000000,141.421356,114.869835,322.0,403.0,259.0,349.0,602.0,2
3,800.0,282.842712,168.179283,123.114441,453.0,519.0,400.0,534.0,833.0,3
4,1600.0,400.000000,200.000000,131.950791,655.0,594.0,500.0,684.0,977.0,4


In [55]:
quick_plot(pd_sync_timelines_w_slope.day,
           pd_sync_timelines_w_slope.iloc[:,1:9],
           y_scale='log',
           slider=True,
           xname = 'Time in days',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Corona infection rate of different countries'           
          )

In [56]:
pd_sync_timelines_w_slope.to_csv('../data/processedCOVID_small_sync_timeline_table.csv',sep=';',index=False)

In [57]:
from sklearn import linear_model
reg  = linear_model.LinearRegression(fit_intercept=False)

In [58]:
l_vec = len(df_analyse['Italy'])
X=np.arange(l_vec-10).reshape(-1, 1) # Feature(training) 
y=np.log(np.array(df_analyse['Italy'][10:])) #Targets(training)

In [59]:
reg.fit(X,y) # Train the model

LinearRegression(fit_intercept=False)

In [60]:
X_hat = np.arange(l_vec).reshape(-1, 1) #Feature(test)
y_hat = reg.predict(X_hat) # Prediction

In [61]:
LR_inspect = df_analyse[['Italy','date']].copy()
LR_inspect['prediction'] = np.exp(y_hat)
LR_inspect.head()

,Italy,date,prediction
0,0,2020-01-22,1.000000
1,0,2020-01-23,1.026713
2,0,2020-01-24,1.054139
3,0,2020-01-25,1.082298
4,0,2020-01-26,1.111209


In [62]:
fig = go.Figure()
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,::2],
           y_scale='log',
           slider=True,
           xname = 'date',
           yname = 'Infected people (source: John Hopkins csse) (log scale)',
           figname = 'Linear regression'          
          )

# Doubling Rate - Piecewise Linear Regression

In [63]:
from sklearn import linear_model
from scipy import signal

In [64]:
reg  = linear_model.LinearRegression(fit_intercept=True)

In [65]:
l_vec = len(df_analyse['Italy'])
X=np.arange(l_vec-10).reshape(-1, 1)
y=np.array(df_analyse['Italy'][10:])

In [66]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])    
country_list=df_analyse.columns[1:]

In [67]:
filter_cols = []
for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           5, # Window size used for filtering
                           1) # Order of fitted polynomial
    filter_cols.append(each+'_filter')

In [68]:
start_pos=5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:], #['US','US_filter']
           y_scale='log',
           slider=True,
           xname = 'date',
           yname = 'Number of cases (filtered) (log scale)',
           figname = 'Filtered number of cases')

In [69]:
df_analyse.head()

,date,Italy,US,Spain,Germany,"Korea, South",Italy_filter,US_filter,Spain_filter,Germany_filter,"Korea, South_filter"
0,2020-01-22,0,1,0,0,1,0.0,0.4,0.0,0.0,0.8
1,2020-01-23,0,1,0,0,1,0.0,1.3,0.0,0.0,1.3
2,2020-01-24,0,2,0,0,2,0.0,2.2,0.0,0.0,1.8
3,2020-01-25,0,2,0,0,2,0.0,3.0,0.0,0.2,2.4
4,2020-01-26,0,5,0,0,3,0.0,3.8,0.0,1.0,3.0


In [70]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [71]:
def doubling_time(in_array):
    ''' Use a classical doubling time formula, see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [72]:
# Calculate slope of regression of last x days
# Use always a limited number of days to approximate the triangle
days_back = 3  # This gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [73]:
# Run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [74]:
# Cross check the math 
df_analyse['Italy_DR_math']=df_analyse['Italy'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)
df_analyse.columns

Index(['date', 'Italy', 'US', 'Spain', 'Germany', 'Korea, South',
       'Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter',
       'Korea, South_filter', 'Italy_DR', 'US_DR', 'Spain_DR', 'Germany_DR',
       'Korea, South_DR', 'Italy_filter_DR', 'US_filter_DR', 'Spain_filter_DR',
       'Germany_filter_DR', 'Korea, South_filter_DR', 'Italy_DR_math'],
      dtype='object')

In [75]:
start_pos=40
quick_plot(df_analyse.date[start_pos: start_pos+30],
           df_analyse.iloc[start_pos: start_pos+30,[11,12,13,14,15]],
           y_scale='linear',
           slider=True,
           xname = 'date',
           yname = 'Doubling rate',
           figname = 'Doubling rate'
          )

In [76]:
start_pos=40
quick_plot(df_analyse.date[start_pos: start_pos+30],
           df_analyse.iloc[start_pos:start_pos+30, [16,17,18,19,20]],
           y_scale='linear',
           slider=True,
           xname = 'date',
           yname = 'Doubling rate(filtered data)',
           figname = 'Doubling rate for filtered data'
          )